## FESOM Interpolate intro cuadrangular grids

In [1]:
## import 
import numpy as np
import xarray as xr
import xoak
import matplotlib.pyplot as plt
import cmocean
import pyfesom2 as pf


/gxfs_home/geomar/smomw662/.local/lib/python3.11/site-packages/pyfesom2/climatology.py:14: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater as sw


In [3]:
# Load and set up FESOM mesh and diagnostic data
meshpath = '/gxfs_work/geomar/smomw662/fesom_parcels_001/data/channel'
mesh = pf.load_mesh(meshpath, abg=[0, 0, 0], usepickle=False)
ds_mesh = xr.open_dataset(f"{meshpath}/fesom.mesh.diag.nc").set_coords(["lat", "lon"])
ds_mesh = ds_mesh.assign_coords(node_id=(("nod2",), range(ds_mesh.sizes["nod2"])))


Load 2d part of the mesh in 0 second(s)


/gxfs_home/geomar/smomw662/.local/lib/python3.11/site-packages/pyfesom2/load_mesh_data.py:200: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  file_content = pd.read_csv(


In [4]:
ds_mesh

<xarray.Dataset> Size: 9MB
Dimensions:            (nz: 41, nz1: 40, elem: 21120, nod2: 10608, n3: 3,
                        n2: 2, edg_n: 31728, N: 6, n4: 4)
Coordinates:
  * nz                 (nz) float64 328B 0.0 9.038 18.98 ... 3.628e+03 4e+03
  * nz1                (nz1) float64 320B 4.519 14.01 ... 3.459e+03 3.814e+03
    lon                (nod2) float64 85kB ...
    lat                (nod2) float64 85kB ...
    node_id            (nod2) int64 85kB 0 1 2 3 4 ... 10604 10605 10606 10607
Dimensions without coordinates: elem, nod2, n3, n2, edg_n, N, n4
Data variables: (12/19)
    fesom_mesh         int32 4B ...
    elem_area          (elem) float64 169kB ...
    nlevels_nod2D      (nod2) int32 42kB ...
    nlevels            (elem) int32 84kB ...
    nod_in_elem2D_num  (nod2) int32 42kB ...
    nod_part           (nod2) int32 42kB ...
    ...                 ...
    face_links         (n3, elem) float64 507kB ...
    edge_face_links    (n2, edg_n) float64 508kB ...
    nod_in_elem2D      (N, nod2) int32 255kB ...
    edge_cross_dxdy    (n4, edg_n) float64 1MB ...
    gradient_sca_x     (n3, elem) float64 507kB ...
    gradient_sca_y     (n3, elem) float64 507kB ...
Attributes:
    Conventions:  UGRID-1.0

In [5]:
# Set up bounds for interpolating onto a quadratic grid
lon_bds = (ds_mesh.lon.min().data[()], np.ceil(ds_mesh.lon.max().data[()]))
lat_bds = (ds_mesh.lat.min().data[()], np.ceil(ds_mesh.lat.max().data[()]))
lon_grid = xr.DataArray(np.linspace(*lon_bds, 101), dims=("lon_grid",), name="lon_grid")
lat_grid = xr.DataArray(np.linspace(*lat_bds, 101), dims=("lat_grid",), name="lat_grid")
lon_grid_selector, lat_grid_selector = xr.broadcast(lon_grid, lat_grid)


In [6]:
# Set up xoak index for spatial selection
ds_mesh.xoak.set_index(["lat", "lon"], "sklearn_geo_balltree")
nodes_grid = ds_mesh.xoak.sel(lon=lon_grid_selector, lat=lat_grid_selector).node_id
nodes_grid = nodes_grid.rename({"lon": "lon_fesom", "lat": "lat_fesom"}).assign_coords(lon_grid=lon_grid, lat_grid=lat_grid)


In [8]:
# Assuming 'variable_name' is the name of the real data variable in your dataset
#data_fesom = ds_mesh['variable_name']  # Use actual data directly

# Plotting
str_id = 'u'
year = 1958
level = 0
time = -1

dat1 = xr.open_dataset(f'{meshpath}/{str_id}.fesom.{year}.nc')[str_id]
dat2 = dat1.isel(time=time, nz1=level) # may need to change nz1 to nz, depending on quantity you plot
dat = dat2.squeeze()


In [14]:
dat1

<xarray.DataArray 'u' (time: 365, nz1: 40, elem: 21120)> Size: 1GB
[308352000 values with dtype=float32]
Coordinates:
  * nz1      (nz1) float64 320B 4.519 14.01 24.45 ... 3.459e+03 3.814e+03
  * time     (time) datetime64[ns] 3kB 1958-01-01T23:50:00 ... 1958-12-31T23:...
Dimensions without coordinates: elem
Attributes:
    description:  zonal velocity
    long_name:    zonal velocity
    units:        m/s
    location:     face
    mesh:         fesom_mesh

In [16]:
# Interpolate onto grid
data_grid = dat1.isel(nod2=nodes_grid)
dat = data_grid.isel(time=time).squeeze()  # Interpolated data on grid


ValueError: Dimensions {'nod2'} do not exist. Expected one or more of FrozenMappingWarningOnValuesAccess({'nz1': 40, 'time': 365, 'elem': 21120})

In [ ]:
# Set up the element positions in the FESOM mesh for plotting
elem_n = ds_mesh.sizes["elem"]
xx2, yy2 = np.zeros(elem_n), np.zeros(elem_n)
for i in range(elem_n):
    xx2[i] = mesh.x2[mesh.elem[i, :]].mean(axis=0)
    yy2[i] = mesh.y2[mesh.elem[i, :]].mean(axis=0)



In [ ]:

# Determine X and Y coordinates for plot
X, Y = xx2, yy2

In [ ]:
# Plot with color map
vmin, vmax = np.round(dat.min().values), np.round(dat.max().values)
cmap = cmocean.cm.thermal
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.tripcolor(X, Y, dat, shading='flat', cmap=cmap)

# Add color bar and titles
cbar = fig.colorbar(im, orientation='horizontal', pad=.05, extend='both')
cbar.set_label(f'{str_id} / {dat.units}', size=12)
im.set_clim(vmin, vmax)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title(f'{dat.time.values}, level={level}')
plt.show()